<a href="https://colab.research.google.com/github/Tan-Jiahua/migration-time-correction/blob/main/MTcorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Parameters that could be set for migration time correction

time_p_change = 20
# Time for pressure change

max_rsd_of_preliminarily_aligned_peaks = 0.04
# Tolerance of migration time precision of RSD in the alignment for preliminary correction (very few aligned peaks will exceed this limit if parameter 'tolerance_for_merging' is set as a non-zero value)

tolerance_for_merging = 3
# A parameter for slight adjustment after the alignment of preliminary correction. If two consecutive aligned peaks in the same transition are not detected in any identical samples, and the difference of their migration time does not exceed 'tolerance_for_merging' (in minute), they are considered as one peak in preliminary correction.

max_rsd_of_finally_aligned_peaks = 0.02
# Tolerance of migration time precision of RSD in the alignment for final correction (very few aligned peaks will exceed this limit due to re-examination step)

allowed_EORMT = 0.025
# Allowed error of relative migration time (EORMT) during potential marker selection (M1, M2, M3, M4) and re-examination step

In [ ]:
import numpy as np
import pandas as pd
import io

In [ ]:
# Load the CSV file
from google.colab import files
uploaded = files.upload()
# To remove the data, we can use:
# !rm rawdata.csv

Saving rawdata.csv to rawdata.csv


In [ ]:
# Obtain column names of this CSV file
csv_name = list(uploaded.keys())[0]
data = pd.read_csv(csv_name)
data_top = data.head()
column_list = list(data_top.columns)
print(data_top)
print('')
print('Get the column names of this CSV file:')
print(column_list)

   TransitionNum     Q1    Q3     MT        Area     Sample
0              0  115.3  87.1  10.88  11070312.0  01_G1_1_U
1              0  115.3  87.1  13.90  23532902.0  02_G1_2_U
2              0  115.3  87.1  10.60   9611286.0  03_G1_3_U
3              0  115.3  87.1  11.63   6360009.0  04_G2_1_U
4              0  115.3  87.1  12.56  11871461.0  05_G2_2_U

Get the column names of this CSV file:
['TransitionNum', 'Q1', 'Q3', 'MT', 'Area', 'Sample']


In [ ]:
# General functions

# Obtain the largest transition number in all samples
def get_max_transition_num(dfs):
    max_transition_num=0
    for i in range(1,len(dfs)+1):
        if dfs[i]['Transition'].max()>max_transition_num:
            max_transition_num=dfs[i]['Transition'].max()
    return max_transition_num

# Get the number of samples in df format
def get_num_of_samples_for_df(df):
    num_of_samples=0
    while str(num_of_samples+1)+'_time' in df.columns:
        num_of_samples+=1
    return num_of_samples

def get_num_of_peaks_in_a_row(df):
    peak_no=1
    num_of_peaks=0
    while str(peak_no)+'_time' in df.index:
        if pd.notnull(df[str(peak_no)+'_time']):
            num_of_peaks+=1
        peak_no+=1
    return num_of_peaks

#Attach average migration time and its relative standard deviation
#If there is corrected migration time values named 'n_time_6markers' or 'n_time_4markers' e. g. '1_time_6markers', the corrected values will be used in calculation. Otherwise, the original migration time will be used.
def with_avgMT_and_RSD(df_input):
    df=df_input.copy()
    is_Series=False
    if isinstance(df,pd.Series):
        df=df.to_frame().T
        is_Series=True
    num_of_samples=get_num_of_samples_for_df(df)
    df_time=df.drop(df.columns,axis='columns')
    df_stat_value=df.drop(df.columns,axis='columns')
    if str(num_of_samples)+'_time_6markers' in df.columns:
        for i in range(0,num_of_samples):
            df_time[str(i+1)+'_time_6markers']=df[str(i+1)+'_time_6markers']
    elif str(num_of_samples)+'_time_4markers' in df.columns:
        for i in range(0,num_of_samples):
            df_time[str(i+1)+'_time_4markers']=df[str(i+1)+'_time_4markers']
    else:
        for i in range(0,num_of_samples):
            df_time[str(i+1)+'_time']=df[str(i+1)+'_time']
    df['Average_MT']=df_time.mean(axis=1)
    df['std']=df_time.std(axis=1,ddof=0)
    df['RSD(%)']=df['std']/df['Average_MT']*100
    df=df.drop(['std'],axis=1)
    if is_Series:
        df=df.squeeze()
    return df

#Select 'idx' representing a peak in 'transition' labelled as 'label'
#If the peak does not exist in variable 'chart_from_idx_to_transition_and_label_inIntermediateStage', return -1
def transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,transition,label):
    num_of_idx=chart_from_idx_to_transition_and_label_inIntermediateStage.shape[0]
    idx=0
    while idx!=num_of_idx \
        and (chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][idx]!=transition or \
        chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][idx]!=label):
        idx+=1
    if idx==num_of_idx:
        print('Peak '+str(transition)+label+' is not in variable dfs')
        return -1
    return idx

In [ ]:
# Function 1: process the uploaded CSV file and return the individual dataset for each sample
def F1_create_datasets(csv_name, column_name_for_transition, column_name_for_precursor_ion, 
                   column_name_for_product_ion, column_name_for_time, 
                   column_name_for_area, column_name_for_sample):
    csv_file = csv_name
    sample_column = column_name_for_sample
    area_column = column_name_for_area
    time_column = column_name_for_time
    Q1 = column_name_for_precursor_ion
    Q3 = column_name_for_product_ion
    transition_column = column_name_for_transition
    data = io.BytesIO(uploaded[csv_name]) 
    df_input = pd.read_csv(data)
    df = df_input.fillna(0) 
    df_no_zero = df[df[area_column]>0].reset_index(drop=True)
    sample_name = df_no_zero[sample_column].unique()
    print('There are {} samples.'.format(len(sample_name)))
    print('')
    print('The peak numbers of each sample are as follow:')
    print(df_no_zero[sample_column].value_counts())
    print('')
    
    sample_number = len(sample_name)
    df_no_zero['sample'] = df_no_zero[sample_column].tolist()
    #df_no_zero['sample'] = df_no_zero[sample_column].apply(lambda x:x[:2]).tolist()
    df2 = df_no_zero.drop([sample_column], axis=1)
    dfs={}
    types={}
    i=0

    for i in range(sample_number):
        # i_str = str(i+1)       
        # i_str_2digits = i_str.zfill(2)
        # print('The dataset of sample {} is created.'.format(i + 1))
        # dfs[i + 1] = df2.groupby(['sample']).get_group(i_str_2digits).reset_index(drop=True)
        dfs[i + 1] = df2[df2['sample'] == sample_name[i]] 
        dfs[i + 1] = dfs[i + 1].drop(columns=[Q1, Q3,'sample'])
        dfs[i + 1] = dfs[i + 1].rename(columns={transition_column:'Transition',
                                        time_column: str(i+1)+'_time', 
                                        area_column: str(i+1)+'_area'}).reset_index(drop=True)
    print('')
    print('Function 1 is completed. Datasets can be viewed by using "print(dfs)" or "print(dfs[i].head())".')
    return dfs

In [ ]:
# Using Function 1
column_name_for_transition = column_list[0]
column_name_for_precursor_ion = column_list[1]
column_name_for_product_ion = column_list[2]
column_name_for_time = column_list[3]
column_name_for_area = column_list[4]
column_name_for_sample = column_list[5]

raw_data = F1_create_datasets(csv_name, column_name_for_transition, column_name_for_precursor_ion,
                         column_name_for_product_ion, column_name_for_time,
                         column_name_for_area, column_name_for_sample)

There are 32 samples.

The peak numbers of each sample are as follow:
14_G5_2_U    381
29_G2_3_R    357
23_G3_3_S    340
22_G3_2_S    339
32_G4_3_R    336
28_G2_2_R    335
31_G4_2_R    329
27_G2_1_R    319
08_G3_2_U    317
17_G1_3_S    310
01_G1_1_U    293
16_G1_2_S    283
21_G3_1_S    281
09_G3_3_U    275
30_G4_1_R    272
06_G2_3_U    272
15_G1_1_S    268
02_G1_2_U    262
05_G2_2_U    258
07_G3_1_U    258
10_G4_1_U    249
24_G4_1_S    249
18_G2_1_S    246
13_G5_1_U    245
04_G2_1_U    242
19_G2_2_S    234
25_G4_2_S    226
12_G4_3_U    220
03_G1_3_U    219
11_G4_2_U    207
26_G4_3_S    192
20_G2_3_S    173
Name: Sample, dtype: int64


Function 1 is completed. Datasets can be viewed by using "print(dfs)" or "print(dfs[i].head())".


In [ ]:
# Check one of the sample datasets
# print(raw_data[32].head(20))

In [ ]:
#General functions

#Check and remove unqualified peak based on allowed EORMT value
#df_input with index named by 'Transition', ('Transition','Label') or None are acceptable
#Average migration time value is required in df_input (should be prepocessesed by function with_avgMT_and_RSD)

def check_and_correct(df_input,allowed_EORMT,location):
    df=df_input.copy()

    if '1_time_6markers' in df.columns:
        time_suffix='_time_6markers'
    elif '1_time_4markers' in df.columns:
        time_suffix='_time_4markers'
    else:
        time_suffix='_time'

    axes_name=df.axes[0].name#When index is 'Transition', this variable will be called later
    axes_names=df.axes[0].names#When index is ('Transition,'Label'), this variable will be called later
    df=df.sort_values(by='Average_MT').reset_index()

    if location=='step_2': 
        df['remove']=False
    else:#only available when 
        df['earlyList']=''
        df['earlyList']=df['earlyList'].apply(list)
        df['lateList']=''
        df['lateList']=df['lateList'].apply(list)

    num_of_samples=get_num_of_samples_for_df(df)
    for j in range(1,num_of_samples+1):
        if df[str(j)+time_suffix].count()<=3:
            continue

        #The first peak is analyzed at first
        peak1_is_first_peak=True
        
        for i in range(0,df.shape[0]-1):
            t_peak1 = float(df[str(j)+time_suffix][i])
            if pd.isnull(t_peak1):
                continue
            if peak1_is_first_peak: #similar to condition "i==0" in previous version
                idx2=i+1
                while df.loc[idx2].isnull().any() or (location!='step_2' and df['RSD(%)'][idx2]>1.5):
                    idx2+=1
                idx3=idx2+1
                while df.loc[idx3].isnull().any() or (location!='step_2' and df['RSD(%)'][idx3]>1.5):
                    idx3+=1
                if not df.loc[i].isnull().any() or (location!='step_2' and df['RSD(%)'][i]>1.5):
                    peak1_is_first_peak=False #Other peaks are no longer the first available one
            else:
                idx2=i-1
                while df.loc[idx2].isnull().any() or (location!='step_2' and df['RSD(%)'][idx2]>1.5):
                    idx2-=1
                idx3=i+1
                peak1_is_last_peak=False # assume that peak 1 is not the last peak
                
                #In case idx3 exceeds the range of df
                #That means peak 1 is the last peak
                if idx3>=df.shape[0]: 
                    peak1_is_last_peak=True
                while df.loc[idx3].isnull().any() or (location!='step_2' and df['RSD(%)'][idx3]>1.5):
                    idx3+=1

                    #When idx3 can exceed the range of index of df, peak 1 is
                    #the last peak
                    if idx3>=df.shape[0]:
                        peak1_is_last_peak=True
                        break
                
                #This part is equavalent to the procession for the last peak,
                #and it can also prevent the error when the last peak is not
                #labelled as the last index
                if peak1_is_last_peak:
                    idx3=idx2-1
                    while df.loc[idx3].isnull().any() or (location!='step_2' and df['RSD(%)'][idx3]>1.5):
                        idx3-=1

            t_peak2 = float(df[str(j)+time_suffix][idx2])
            t_peak3 = float(df[str(j)+time_suffix][idx3])
            t_peak1_average_of_all_samples = float(df['Average_MT'][i])
            t_peak2_average_of_all_samples = float(df['Average_MT'][idx2])
            t_peak3_average_of_all_samples = float(df['Average_MT'][idx3])
            EORMT1 = abs((t_peak1/t_peak2)/(t_peak1_average_of_all_samples/t_peak2_average_of_all_samples)-1)
            EORMT2 = abs((t_peak1/t_peak3)/(t_peak1_average_of_all_samples/t_peak3_average_of_all_samples)-1)
            if EORMT1>=allowed_EORMT and EORMT2>=allowed_EORMT:
                if 'Transition' in df.columns and 'Label' not in df.columns:
                    print('Transition {} in sample {} is abnormal, EORMT is {:.2%}>2.5% to the left peak (in transition {}) and {:.2%}>2.5% to the right peak (in transition {}).'.format(
                        df.loc[i,'Transition'],j,EORMT1,df.loc[idx2,'Transition'],EORMT2,df.loc[idx3,'Transition']))
                elif 'Transition' in df.columns and 'Label' in df.columns:
                    print('Peak {}{} in sample {} is abnormal, EORMT is {:.2%}>2.5% to the left peak ({}{}) and {:.2%}>2.5% to the right peak ({}{}).'.format(
                        df.loc[i,'Transition'],df.loc[i,'Label'],j,EORMT1,df.loc[idx2,'Transition'],df.loc[idx2,'Label'],EORMT2,df.loc[idx3,'Transition'],df.loc[idx3,'Label']))
                if location=='step_2':
                    df.loc[i,'remove']=True
                else:
                    if t_peak1<t_peak1_average_of_all_samples:
                        df.loc[i,'earlyList'].append(j)
                    else:
                        df.loc[i,'lateList'].append(j)

    if location=='step_2':    
        df=df[df['remove']==False]
        df=df.drop('remove',axis='columns')
    else:
        idx=0
        df_preupdate=df.sort_values(by=['Transition','Label'])
        min_transition=min(df_preupdate['Transition'])
        max_transition=max(df_preupdate['Transition'])

        #Edit transition No. i
        for i in range(min_transition,max_transition+1):
            print('Processing peaks in Transition '+str(i)+'.')
            peaks_in_one_transition=df_preupdate[df_preupdate['Transition']==i]
            peaks_in_one_transition=peaks_in_one_transition.reset_index(drop=True)
            num_of_peaks_in_one_transition=peaks_in_one_transition.shape[0]
            if num_of_peaks_in_one_transition==0:
                continue
            idx=0
            update_label=0
            peaks_in_one_transition_update_not_exist=True

            for idx in range(0,num_of_peaks_in_one_transition):
                analyzed_peak=peaks_in_one_transition.iloc[idx]

                if get_num_of_peaks_in_a_row(analyzed_peak)/num_of_samples<0.8:
                    can_recombine=True
                else:
                    can_recombine=False

                # Initialize variable
                peaks_earlyList=pd.DataFrame({'Transition':[i],'Label':np.nan})#Label temporarily unavailable
                peaks_normalList=pd.DataFrame({'Transition':[i],'Label':np.nan})#Label temporarily unavailable
                peaks_lateList=pd.DataFrame({'Transition':[i],'Label':np.nan})#Label temporarily unavailable
                analyzed_peak_update_not_exist=True
                normalListNotNull=False

                for j in range(1,num_of_samples+1):
                    peaks_earlyList[str(j)+'_time']=np.nan
                    peaks_earlyList[str(j)+'_area']=np.nan
                    peaks_normalList[str(j)+'_time']=np.nan
                    peaks_normalList[str(j)+'_area']=np.nan
                    peaks_lateList[str(j)+'_time']=np.nan
                    peaks_lateList[str(j)+'_area']=np.nan
                    if time_suffix!='_time':
                        peaks_earlyList[str(j)+time_suffix]=np.nan
                        peaks_normalList[str(j)+time_suffix]=np.nan
                        peaks_lateList[str(j)+time_suffix]=np.nan
                    if j in analyzed_peak['earlyList']:
                        peaks_earlyList[str(j)+'_time']=analyzed_peak[str(j)+'_time']
                        peaks_earlyList[str(j)+'_area']=analyzed_peak[str(j)+'_area']
                        if time_suffix!='_time':
                            peaks_earlyList[str(j)+time_suffix]=analyzed_peak[str(j)+time_suffix]
                    elif j in analyzed_peak['lateList']:
                        peaks_lateList[str(j)+'_time']=analyzed_peak[str(j)+'_time']
                        peaks_lateList[str(j)+'_area']=analyzed_peak[str(j)+'_area']
                        if time_suffix!='_time':
                            peaks_lateList[str(j)+time_suffix]=analyzed_peak[str(j)+time_suffix]
                    else:
                        if pd.notnull(analyzed_peak[str(j)+'_time']):
                            normalListNotNull=True
                        peaks_normalList[str(j)+'_time']=analyzed_peak[str(j)+'_time']
                        peaks_normalList[str(j)+'_area']=analyzed_peak[str(j)+'_area']
                        if time_suffix!='_time':
                            peaks_normalList[str(j)+time_suffix]=analyzed_peak[str(j)+time_suffix]
   
                #analyzed_peak will not be empty
                if len(analyzed_peak['earlyList'])!=0:
                    update_label+=1
                    peaks_earlyList['Label']=chr(update_label+64)
                    analyzed_peak_update=peaks_earlyList
                    analyzed_peak_update_not_exist=False
                if normalListNotNull:
                    update_label+=1
                    peaks_normalList['Label']=chr(update_label+64)
                    if analyzed_peak_update_not_exist:
                        analyzed_peak_update=peaks_normalList
                        analyzed_peak_update_not_exist=False
                    else:
                        analyzed_peak_update=pd.concat([analyzed_peak_update,peaks_normalList])
                if len(analyzed_peak['lateList'])!=0:
                    update_label+=1
                    peaks_lateList['Label']=chr(update_label+64)
                    if analyzed_peak_update_not_exist:
                        analyzed_peak_update=peaks_lateList
                    else:
                        analyzed_peak_update=pd.concat([analyzed_peak_update,peaks_lateList])
                
                analyzed_peak_update['RSD_recombine']=can_recombine
                
                if peaks_in_one_transition_update_not_exist:
                    peaks_in_one_transition_update=analyzed_peak_update
                    peaks_in_one_transition_update_not_exist=False
                else:
                    peaks_in_one_transition_update=pd.concat([peaks_in_one_transition_update,analyzed_peak_update])

            if i==min_transition:
                df_without_RSD_correction=peaks_in_one_transition_update
            else:
                df_without_RSD_correction=pd.concat([df_without_RSD_correction,peaks_in_one_transition_update])

            '''Modification Step: Modify previous modification: if two set of peaks are consecutive 
            in one transition and both of these peaks do not appear in a single sample 
            simutaneously, a RSD < 2.5 % criterion will be adopted to recombine the result'''
            j=0
            num_of_lines=peaks_in_one_transition_update.shape[0]
            while j<num_of_lines-1:
                metabolite_checking=with_avgMT_and_RSD(peaks_in_one_transition_update.iloc[j])
                if j!=0:
                    metabolite_prev=with_avgMT_and_RSD(peaks_in_one_transition_update.iloc[j-1])
                    mergeable_prev=metabolite_prev['RSD_recombine'] and metabolite_checking['RSD_recombine']
                    '''mergeable_prev=(get_num_of_peaks_in_a_row(metabolite_prev)/num_of_samples<0.8) and \
                        (get_num_of_peaks_in_a_row(metabolite_checking)/num_of_samples<0.8)'''
                else:
                    mergeable_prev=False
                if j!=num_of_lines:                    
                    metabolite_post=with_avgMT_and_RSD(peaks_in_one_transition_update.iloc[j+1])
                    mergeable_post=metabolite_post['RSD_recombine'] and metabolite_checking['RSD_recombine']
                    '''mergeable_post=(get_num_of_peaks_in_a_row(metabolite_post)/num_of_samples<0.8) and \
                        (get_num_of_peaks_in_a_row(metabolite_checking)/num_of_samples<0.8)'''
                else:
                    mergeable_post=False

                for label_use_no in range(1,num_of_samples+1):
                    label=str(label_use_no)+time_suffix
                    if mergeable_prev and pd.notnull(metabolite_prev[label]) and \
                        pd.notnull(metabolite_checking[label]):
                        mergeable_prev=False
                    if mergeable_post and pd.notnull(metabolite_post[label]) and \
                        pd.notnull(metabolite_checking[label]):
                        mergeable_post=False

                if mergeable_prev and mergeable_post:
                    if metabolite_post['Average_MT']-metabolite_checking['Average_MT']>\
                        metabolite_checking['Average_MT']-metabolite_prev['Average_MT']:
                        mergeable_post=False
                    else:
                        mergeable_prev=False

                #Trial of mergeable sets of peaks
                if mergeable_prev:
                    trial_merged_peaks=with_avgMT_and_RSD(metabolite_prev.add(metabolite_checking,fill_value=0))
                    if trial_merged_peaks['RSD(%)']<2.5:
                        peaks_in_one_transition_update['Label']=peaks_in_one_transition_update['Label'].map(lambda l:chr(ord(l)-1) if ord(l)>=j+65 else l)
                        j-=1

                if mergeable_post:
                    trial_merged_peaks=with_avgMT_and_RSD(metabolite_post.add(metabolite_checking,fill_value=0))
                    if trial_merged_peaks['RSD(%)']<2.5:
                        peaks_in_one_transition_update['Label']=peaks_in_one_transition_update['Label'].map(lambda l:chr(ord(l)-1) if ord(l)>j+65 else l)
                
                if mergeable_prev or mergeable_post:
                        peaks_in_one_transition_update=peaks_in_one_transition_update.groupby('Label',sort=False).max()
                        peaks_in_one_transition_update=peaks_in_one_transition_update.reset_index()
                        peaks_in_one_transition_update=peaks_in_one_transition_update.set_index(['Transition','Label'])
                        peaks_in_one_transition_update=peaks_in_one_transition_update.reset_index()
                        num_of_lines-=1
                        
                j+=1

            if i==min_transition:
                df=peaks_in_one_transition_update
            else:
                df=pd.concat([df,peaks_in_one_transition_update])

    if axes_name!=None: #The most common example: 'Transition' as index
        df=df.set_index(axes_name)
    elif axes_names!=[None]: #The most common example: ('Transition','Label') as index
        df=df.set_index(axes_names)
    else: #No previous index
        df=df.reset_index(drop=True)
        df.index=df.index+1

    if location=='step_2':
        return df
    else:
        if axes_name!=None: #The most common example: 'Transition' as index
            df_without_RSD_correction=df_without_RSD_correction.set_index(axes_name)
        elif axes_names!=[None]: #The most common example: ('Transition','Label') as index
            df_without_RSD_correction=df_without_RSD_correction.set_index(axes_names)
        else: #No previous index
            df_without_RSD_correction=df_without_RSD_correction.reset_index(drop=True)
            df_without_RSD_correction.index=df_without_RSD_correction.index+1
        
        df_without_RSD_correction=df_without_RSD_correction.drop('RSD_recombine',axis='columns')
        df=df.drop('RSD_recombine',axis='columns')    
        return df

In [ ]:
# Function 2: Suggest the potential migration time markers for Stage 1 and Stage 2 
# based on three conditions: 1. only one peak is found in that transition(Q1→Q3);
# 2. this analyte can be found in all samples; 3. this analyte should only be found
# either in Stage 1 or Stage 2.

# If time_p_chnage is set as a double value, the removal part will be run
# If time_p_chnage is set as False, the removal part will not be run

def F2_find_markers(dfs,time_p_change):
    # Get the number of transitions from a sample set
    num_of_transitions=get_max_transition_num(dfs)
    one_peak=pd.DataFrame({'contains_one_peak':[True]*(num_of_transitions+1)},
                          index=range(0,(num_of_transitions+1)))

    # Find the one-peak transition
    for i in range(1,len(dfs)+1):
        num_is_one=dfs[i].groupby(['Transition'])['Transition'].count()==1
        for j in range(0,num_of_transitions+1):
            if j not in num_is_one.index or not num_is_one[j]:
                one_peak['contains_one_peak'][j]=False
    selected=one_peak.loc[one_peak['contains_one_peak']].index
    df_time_area=dfs[1].set_index('Transition').loc[selected]
    for i in range(2,len(dfs)+1):
        right=dfs[i].set_index('Transition').loc[selected]
        df_time_area=df_time_area.join(right)

    # Review all analytes migration times and peak areas
    print('All {} one-peak transitions were selected.'.format(df_time_area.shape[0]))
    print('')
    df_time=df_time_area
    for i in range(1,len(dfs)+1):
        df_time=df_time.drop(str(i)+'_area',axis=1)
    df_time=with_avgMT_and_RSD(df_time)
    df_time.insert(0,'Transition',df_time.index)
    df_time=df_time.sort_values(by='Average_MT').reset_index(drop=True) #Perhaps can be deleted

    # If time_p_chnage is set as a double value, the following part will be run
    # If time_p_chnage is set as False, the following part will not be run
    if time_p_change:
        # Remove peaks in the Intermediate Stage
        print('The following transitions will be removed, because analytes in these transistions were detected in both phase 1 and phase 2.')
        for i in range(0,len(df_time.index)):
            times = df_time.loc[i,:]
            times = times.drop(['Transition','Average_MT','RSD(%)'])
            times_max = times.max(axis=0)
            times_min = times.min(axis=0)
            avg = float(df_time['Average_MT'][i])
            if times_max > time_p_change and times_min < time_p_change:
                print('The transition {} is removed.'.format(int(df_time.loc[i, ['Transition']])))
                df_time = df_time.drop([i])
        df_time=df_time.reset_index(drop=True)
        df_time.index=df_time.index+1
        print('')

        # Only keep the one-analyte transistion by checking the migration order of each peak with its adjacent peaks.
        df_stage1 = df_time[df_time['Average_MT'] < time_p_change].reset_index(drop=True).copy()
        df_stage2 = df_time[df_time['Average_MT'] >= time_p_change].reset_index(drop=True).copy()
        print('The relative position for each peak will be checked according to its two adjacent peaks.')
        print('Start to check peaks in Stage 1:')
        df_stage1=check_and_correct(df_stage1,allowed_EORMT,'step_2')
        print('Peaks in Stage 1 have been checked.')

        print('Start to check peaks in Stage 2:')
        df_stage2=check_and_correct(df_stage2,allowed_EORMT,'step_2')
        print('Peaks in Stage 2 have been checked.')
        print('The peaks above were removed because they may be assigned to more than 1 analyte.')

        df_time=pd.concat([df_stage1,df_stage2])
    else:
        print('No peaks are checked and removed, for variable "check_and_correct" is set as False.')
    print('')
    df_time=df_time.reset_index(drop=True)
    df_time.index=df_time.index+1
    print('Function 2 is completed.')
    print('')

    return df_time

In [ ]:
# Using Function 2
# Apply Function 2 to find potencial migration time markers
pd.set_option('precision', 2)
potential_markers = F2_find_markers(raw_data, time_p_change)
print('Migration time markers for Stage 1 and Stage 2 may be selected from the following table:')
print('')
print(potential_markers)

All 32 one-peak transitions were selected.

The following transitions will be removed, because analytes in these transistions were detected in both phase 1 and phase 2.
The transition 147 is removed.
The transition 230 is removed.
The transition 262 is removed.
The transition 260 is removed.
The transition 258 is removed.

The relative position for each peak will be checked according to its two adjacent peaks.
Start to check peaks in Stage 1:
Transition 125 in sample 1 is abnormal, EORMT is 27.36%>2.5% to the left peak (in transition 27) and 26.14%>2.5% to the right peak (in transition 14).
Transition 14 in sample 1 is abnormal, EORMT is 20.72%>2.5% to the left peak (in transition 125) and 5.48%>2.5% to the right peak (in transition 60).
Transition 125 in sample 3 is abnormal, EORMT is 6.43%>2.5% to the left peak (in transition 27) and 4.43%>2.5% to the right peak (in transition 14).
Transition 125 in sample 4 is abnormal, EORMT is 4.97%>2.5% to the left peak (in transition 27) and 4.2

In [ ]:
potential_markers.drop('index', axis = 'columns')
# Based on the following results, we firstly selected the analytes in transitions 38 (1-methylnicotinamide) and 24 (taurine)
# as the markers for Stage 1 , and the analytes in transitions 182 (N-acetyltryptophan) and 98 (hippuric acid) as the markers for Stage 2.

,Transition,1_time,2_time,3_time,4_time,5_time,6_time,7_time,8_time,9_time,10_time,11_time,12_time,13_time,14_time,15_time,16_time,17_time,18_time,19_time,20_time,21_time,22_time,23_time,24_time,25_time,26_time,27_time,28_time,29_time,30_time,31_time,32_time,Average_MT,RSD(%)
1,253,4.83,5.30,4.58,4.86,5.08,5.14,5.27,4.86,5.05,5.24,5.14,5.14,5.21,4.89,4.86,4.43,5.01,5.26,5.43,5.18,5.08,4.89,5.36,4.49,4.68,5.11,5.39,4.86,5.24,5.25,5.14,5.31,5.05,5.04
2,37,5.49,5.92,5.02,5.45,5.61,5.77,5.92,5.55,5.58,5.99,5.77,5.61,5.77,5.58,5.49,5.08,5.70,6.02,6.27,5.71,5.83,5.55,6.05,4.99,5.30,5.76,5.95,5.61,5.92,5.83,5.80,5.89,5.68,5.10
3,38,5.61,6.27,5.30,5.67,5.86,6.20,6.30,5.77,5.89,6.27,6.08,5.95,6.05,5.86,5.86,5.24,5.92,6.33,6.55,5.95,6.08,5.80,6.27,5.21,5.58,6.02,6.23,5.80,6.20,6.20,6.05,6.20,5.96,5.34
4,27,6.67,7.57,6.36,6.86,7.23,7.42,7.64,6.83,7.14,7.51,7.36,7.29,7.33,6.89,6.95,6.30,7.14,7.64,7.92,7.26,7.39,6.86,7.54,6.27,6.76,7.26,7.57,6.92,7.48,7.39,7.26,7.51,7.17,5.62
5,60,10.63,13.34,10.13,11.16,12.10,12.10,12.94,11.10,11.44,13.28,12.56,12.06,12.47,11.19,11.78,9.98,11.38,13.34,13.75,12.22,12.47,11.16,13.28,10.04,11.28,12.22,12.84,11.19,12.09,12.19,12.25,12.19,11.94,8.18
6,3,10.63,13.34,10.13,11.22,12.06,12.13,12.94,11.10,11.44,13.28,12.56,12.06,12.50,11.22,11.78,9.98,11.38,13.37,13.72,12.22,12.50,11.13,13.28,10.04,11.28,12.22,12.84,11.22,12.09,12.22,12.25,12.22,11.95,8.17
7,36,10.69,13.50,10.32,11.25,12.19,12.19,13.06,11.16,11.57,13.37,12.62,12.19,12.56,11.29,11.91,10.10,11.50,13.53,13.87,12.28,12.59,11.25,13.37,10.10,11.35,12.28,12.94,11.25,12.16,12.41,12.34,12.28,12.05,8.17
8,39,10.72,13.72,10.41,11.47,12.38,12.50,13.59,11.38,11.75,13.65,12.94,12.50,12.78,11.47,12.06,10.29,11.75,13.72,14.03,12.44,13.12,11.47,13.65,10.22,11.50,12.47,13.40,11.44,12.56,12.66,12.66,12.66,12.29,8.49
9,163,10.88,13.87,10.60,11.63,12.56,12.62,13.72,11.44,11.88,13.87,13.15,12.69,12.97,11.72,12.31,10.38,12.03,13.93,14.31,12.63,13.22,11.60,13.81,10.38,11.63,12.66,13.53,11.63,12.81,12.87,12.75,12.94,12.47,8.47
10,161,10.88,13.87,10.57,11.63,12.53,12.62,13.75,11.44,11.91,13.84,13.15,12.66,12.94,11.72,12.31,10.38,12.03,14.00,14.40,12.66,13.22,11.56,13.84,10.38,11.63,12.66,13.53,11.69,12.84,12.87,12.84,12.94,12.48,8.55


In [ ]:
# Please select markers M1 ~ M4 for following migration time correction
m1_transition = 38
m2_transition = 24
m3_transition = 182
m4_transition = 98

In [ ]:
# Function 3: Calculate γ according to the selected markers
def get_gamma(dfs, sample_num, df_std, m1_transition,m2_transition):
    df_std_temp = df_std.set_index('Transition')
    # Using the average MT as the standard MT
    tm1_std = df_std_temp.loc[m1_transition,'Average_MT'] 
    tm2_std = df_std_temp.loc[m2_transition,'Average_MT']
    df_temp = dfs[sample_num].set_index('Transition')
    tm1 = df_temp.loc[m1_transition,str(sample_num)+'_time']
    tm2 = df_temp.loc[m2_transition,str(sample_num)+'_time']
    gamma = ((1/tm1)-(1/tm2))/((1/tm1_std)-(1/tm2_std))
    return gamma

In [ ]:
# This is an example using Function 3
sample_num = 19
gamma_stage1 = get_gamma(raw_data, sample_num, potential_markers, m1_transition, m2_transition)
gamma_stage2 = get_gamma(raw_data, sample_num, potential_markers, m3_transition, m4_transition)
# print('For sample '+str(sample_num)+', the γ1 is '+str(gamma_stage1)+' and the γ2 is '+str(gamma_stage2))

In [ ]:
# Function 4: Migration time correction by 4 markers
# If migration times cannot be corrected by these markers in case that gamma equals 0, return -1

# time_p_change can be either assigned as the time of pressure change (a double value), or 'False'
# If time_p_change is set as a double value, preliminary correction will be carried out. Parameters 'm1_transition', 'm2_transition', 'm3_transition' and 'm4_transition' should be assigned as the transition where the four markers used in preliminary correction are observed
# If time_p_change is set as False, three gamma values calculated from four peaks will be used. This setting is used during the enumeration process in Function 6.
def correcting_by4markers(dfs_input,df_std,m1_transition,m2_transition,m3_transition,m4_transition,time_p_change):

    #To avoid modifying input parameter
    dfs=dfs_input.copy()
    
    df_std_temp = df_std.set_index('Transition')
    tm1_std = df_std_temp.loc[m1_transition,'Average_MT']
    tm2_std = df_std_temp.loc[m2_transition,'Average_MT']
    tm3_std = df_std_temp.loc[m3_transition,'Average_MT']
    for i in range(1,len(dfs)+1):
        gamma1 = get_gamma(dfs, i, df_std, m1_transition, m2_transition)
        gamma2 = get_gamma(dfs, i, df_std, m3_transition, m4_transition)
        if gamma1==0:
            print('Migration times cannot be corrected by m1_transition='+str(m1_transition)+\
                ' and m2_transition='+str(m2_transition)+', because their migration '+\
                'times in sample '+str(i)+' are identical.')
            return -1
        if gamma2==0:
            print('Migration times cannot be corrected by m3_transition='+str(m3_transition)+\
                  ' and m4_transition='+str(m4_transition)+', because their migration '+\
                  'times in sample '+str(i)+' are identical.')
            return -1
        if not time_p_change:
            gamma_intermediate_stage = get_gamma(dfs, i, df_std, m2_transition, m3_transition)
            if gamma_intermediate_stage==0:
                print('Migration times cannot be corrected by m2_transition='+str(m2_transition)+\
                    ' and m3_transition='+str(m3_transition)+', because their migration '+\
                    'times in sample '+str(i)+' are identical.')
                return -1
        df_temp = dfs[i].set_index('Transition')
        tm1 = df_temp.loc[m1_transition,str(i)+'_time']
        tm2 = df_temp.loc[m2_transition,str(i)+'_time']
        tm3 = df_temp.loc[m3_transition,str(i)+'_time']
        dfs[i][str(i)+'_time_4markers']= None
        analyte_num = dfs[i].shape[0]
        for j in range(0,analyte_num):
            t_origin = dfs[i].loc[j,str(i)+'_time']
            if time_p_change:
                if t_origin <= time_p_change:
                    t_4marker_p = 1/((1/tm1_std)-(1/gamma1)*((1/tm1)-(1/t_origin)))
                else:
                    t_4marker_p = 1/((1/tm3_std)-(1/gamma2)*((1/tm3)-(1/t_origin)))
            else:
                if t_origin <= tm2:
                    t_4marker_p = 1/((1/tm1_std)-(1/gamma1)*((1/tm1)-(1/t_origin)))
                elif t_origin <= tm3:
                    t_4marker_p = 1/((1/tm2_std)-(1/gamma_intermediate_stage)*((1/tm2)-(1/t_origin)))
                else:
                    t_4marker_p = 1/((1/tm3_std)-(1/gamma2)*((1/tm3)-(1/t_origin)))
            dfs[i].loc[j,str(i)+'_time_4markers'] = t_4marker_p
    return dfs

In [ ]:
# Apply Function 4 for preliminary correction
preliminarily_corrected_peaks=correcting_by4markers(raw_data, potential_markers, m1_transition, m2_transition, m3_transition, m4_transition, time_p_change)

In [ ]:
# This is the migration time of the peaks in one sample after preliminary correction (shown as example)
pd.set_option('precision', 6)
# preliminarily_corrected_peaks[32].head(38)

In [ ]:
#Function 5: Peak alignment
#RSDTolerance: maximum allowed RSD for MT in preliminary alignment step
#concatenateTolerance: maximun allowed range for corrected MT in modification step
#if concatenateTolerance == 0, modification step will not make a difference on the result
def align(corrected_dfs_input,RSDTolerance,concatenateTolerance):

    #To avoid modifying input parameter
    dfs=corrected_dfs_input.copy()
    num = len(dfs)

    if '1_time_6markers' in dfs[1].columns:
        suffix='_time_6markers'
    elif '1_time_4markers' in dfs[1].columns:
        suffix='_time_4markers'
    else:
        print('Time in data has not been corrected')
        return

    for i in range(1,num+1):
        dfs[i]['sample_no']=i;
        if suffix=='_time_6markers':
            dfs[i]=dfs[i].rename(columns={str(i)+'_time':'time',str(i)+'_area':'area',str(i)+'_time_6markers':'time_corrected'})
        elif suffix=='_time_4markers':
            dfs[i]=dfs[i].rename(columns={str(i)+'_time':'time',str(i)+'_area':'area',str(i)+'_time_4markers':'time_corrected'})
    
    num_of_transitions = 0
    for i in range(1,num+1):
        if dfs[i]['Transition'].max()>num_of_transitions:
            num_of_transitions=dfs[i]['Transition'].max()
    transitions=range(0,num_of_transitions+1);
    for i in transitions:
        print("The peaks in transition "+str(i)+" of "+str(num)+" samples have been aligned.")
        # Selected peaks in all samples in single transition
        for no_of_sample,sample in dfs.items():
            peaks_in_one_transition_in_one_sample=sample.loc[sample['Transition']==i]
            if no_of_sample==1 or peaks_in_one_transition.empty:
                peaks_in_one_transition=peaks_in_one_transition_in_one_sample
            else:
                peaks_in_one_transition=pd.concat([peaks_in_one_transition,peaks_in_one_transition_in_one_sample])
        
        # Skip the empty transition
        if peaks_in_one_transition.shape[0]==0:
            continue;

        peaks_in_one_transition=peaks_in_one_transition.sort_values(by='time_corrected')
        peaks_in_one_transition['']=range(0,peaks_in_one_transition.shape[0])
        peaks_in_one_transition=peaks_in_one_transition.set_index('')
        num_of_peaks_in_one_transition=peaks_in_one_transition.shape[0]
        label_of_peak=1
        idx=0
        idx2=0

        # Preliminary alignment step: align the peaks in single transition
        while idx<num_of_peaks_in_one_transition:
            selected_samples=[]
            min_time=peaks_in_one_transition['time_corrected'][idx]
            analyzed_peak=peaks_in_one_transition.iloc[idx]

            # Initialize variable peaks_aligned_as_one_metabolite
            peaks_aligned_as_one_metabolite=pd.DataFrame({'Transition':[i],'Label':[chr(label_of_peak+64)]})
            for j in range(1,num+1):
                peaks_aligned_as_one_metabolite[str(j)+'_time']=np.nan
                peaks_aligned_as_one_metabolite[str(j)+'_area']=np.nan
                peaks_aligned_as_one_metabolite[str(j)+'_time_corrected']=np.nan

            # Select the first few peaks that are not in the same samples (forward-to-backward scheme)
            while analyzed_peak['sample_no'] not in selected_samples:
                selected_samples.append(analyzed_peak['sample_no'])
                idx+=1
                if idx==num_of_peaks_in_one_transition:
                    break
                analyzed_peak=peaks_in_one_transition.iloc[idx]

            # Eliminate peaks to satisfy RSD condition
            while np.std(list(peaks_in_one_transition['time_corrected'][idx2:idx]))/\
            np.mean(list(peaks_in_one_transition['time_corrected'][idx2:idx]))>=RSDTolerance:
                idx-=1

            # Select peaks that can be aligned as one metabolite
            for j in range(idx2,idx):
                analyzed_peak=peaks_in_one_transition.loc[j]
                selected_samples.append(analyzed_peak['sample_no'])
                peaks_aligned_as_one_metabolite[str(analyzed_peak['sample_no'])+'_time']=analyzed_peak['time']
                peaks_aligned_as_one_metabolite[str(analyzed_peak['sample_no'])+'_area']=analyzed_peak['area']
                peaks_aligned_as_one_metabolite[str(analyzed_peak['sample_no'])+'_time_corrected']=analyzed_peak['time_corrected']

            # Prepare to select peaks for another metabolites
            if len(selected_samples)>=1:
                if label_of_peak==1:
                    peaks_aligned_in_one_transition=peaks_aligned_as_one_metabolite
                else:
                    peaks_aligned_in_one_transition=pd.concat([peaks_aligned_in_one_transition,\
                                        peaks_aligned_as_one_metabolite])
                label_of_peak+=1

            idx2=idx

        #If two consecutive aligned peaks in the same transition are not detected in any identical samples, and the difference of their migration time does not exceed 'concatenateTolerance' , they are considered as one peak in preliminary correction.
        j=0
        lines=peaks_aligned_in_one_transition.shape[0]
        while j<lines-1:
            metabolite_1=peaks_aligned_in_one_transition.iloc[j]
            metabolite_2=peaks_aligned_in_one_transition.iloc[j+1]
            metabolite_1.drop(['Transition','Label'])
            metabolite_2.drop(['Transition','Label'])

            # Examine each corrected time to select the maximum and minimum migration time
            max_time=0
            min_time=2**31
            merge=True;
            for label_use_no in range(1,num+1):
                label=str(label_use_no)+'_time_corrected'
                if pd.notnull(metabolite_1[label]) and pd.notnull(metabolite_2[label]):
                    merge=False;
                    break;
                if pd.notnull(metabolite_1[label]):
                    max_time=max(metabolite_1[label],max_time)
                    min_time=min(metabolite_1[label],min_time)
                elif pd.notnull(metabolite_2[label]):
                    max_time=max(metabolite_2[label],max_time)
                    min_time=min(metabolite_2[label],min_time)
            merge=merge and max_time-min_time<=concatenateTolerance
            if merge:
                peaks_aligned_in_one_transition['Label']=peaks_aligned_in_one_transition['Label'].map(lambda l:chr(ord(l)-1) if ord(l)>j+65 else l)
                peaks_aligned_in_one_transition=peaks_aligned_in_one_transition.groupby('Label',sort=False).max()
                peaks_aligned_in_one_transition=peaks_aligned_in_one_transition.reset_index()
                peaks_aligned_in_one_transition=peaks_aligned_in_one_transition.set_index(['Transition','Label'])
                peaks_aligned_in_one_transition=peaks_aligned_in_one_transition.reset_index()
                lines-=1
            j+=1

        peaks_aligned_in_one_transition=peaks_aligned_in_one_transition.reset_index()

        # Modification step: due to weakness of "forward-to-backward" scheme, the last peak in a metabolite will be tried to align in the next metabolite
        for j in range(0,peaks_aligned_in_one_transition.shape[0]-1):
            while True:
                peaks_aligned_in_one_transition=with_avgMT_and_RSD(peaks_aligned_in_one_transition)
                max_MT_in_one_metabolite = 0
                min_MT_in_next_metabolite = 0
                count_this = 0
                count_next = 0
                for k in range(1,num+1):
                    if not pd.isnull(peaks_aligned_in_one_transition.loc[j,str(k)+'_time_corrected']):
                        count_this = count_this + 1
                    if peaks_aligned_in_one_transition.loc[j,str(k)+'_time_corrected']>max_MT_in_one_metabolite:
                        max_MT_in_one_metabolite=peaks_aligned_in_one_transition.loc[j,str(k)+'_time_corrected']
                        peak_with_max_MT=k

                min_MT_in_next_metabolite=2**31
                for k in range(1,num+1):
                    if not pd.isnull(peaks_aligned_in_one_transition.loc[j+1,str(k)+'_time_corrected']):
                        count_next = count_next + 1
                    if peaks_aligned_in_one_transition.loc[j+1,str(k)+'_time_corrected']<min_MT_in_next_metabolite:
                        min_MT_in_next_metabolite=peaks_aligned_in_one_transition.loc[j+1,str(k)+'_time_corrected']

                if pd.isnull(peaks_aligned_in_one_transition.loc[j+1,str(peak_with_max_MT)+'_time_corrected']) and min_MT_in_next_metabolite-max_MT_in_one_metabolite < 0.04 * (peaks_aligned_in_one_transition.loc[j+1,'Average_MT'] - peaks_aligned_in_one_transition.loc[j,'Average_MT']) and count_this < count_next:#min_MT_in_next_metabolite-max_MT_in_one_metabolite < 0.01*peaks_aligned_in_one_transition.loc[j,'Average_MT']:
                    peaks_aligned_in_one_transition.loc[j+1,str(peak_with_max_MT)+'_time']=peaks_aligned_in_one_transition.loc[j,str(peak_with_max_MT)+'_time']
                    peaks_aligned_in_one_transition.loc[j+1,str(peak_with_max_MT)+'_area']=peaks_aligned_in_one_transition.loc[j,str(peak_with_max_MT)+'_area']
                    peaks_aligned_in_one_transition.loc[j+1,str(peak_with_max_MT)+'_time_corrected']=peaks_aligned_in_one_transition.loc[j,str(peak_with_max_MT)+'_time_corrected']
                    peaks_aligned_in_one_transition.loc[j,str(peak_with_max_MT)+'_time']=np.nan
                    peaks_aligned_in_one_transition.loc[j,str(peak_with_max_MT)+'_area']=np.nan
                    peaks_aligned_in_one_transition.loc[j,str(peak_with_max_MT)+'_time_corrected']=np.nan
                else:
                    break

        if i==transitions[0]:
            aligned_dfs=peaks_aligned_in_one_transition
        else:
            aligned_dfs=pd.concat([aligned_dfs,peaks_aligned_in_one_transition])
    aligned_dfs=aligned_dfs.set_index(['Transition','Label'])
    for i in range(1,num+1):
        dfs[i]['sample_no']=i;
        if suffix=='_time_6markers':
            aligned_dfs=aligned_dfs.rename(columns={str(i)+'_time_corrected':str(i)+'_time_6markers'})
        elif suffix=='_time_4markers':
            aligned_dfs=aligned_dfs.rename(columns={str(i)+'_time_corrected':str(i)+'_time_4markers'})
    print("Function 5 is completed.")

    aligned_dfs = aligned_dfs.drop(labels=['Average_MT','RSD(%)'],axis='columns')
    return aligned_dfs

In [ ]:
# Using Function 5 for the alignment after preliminary correction
preliminarily_corrected_and_aligned_peaks = align(preliminarily_corrected_peaks, max_rsd_of_preliminarily_aligned_peaks, tolerance_for_merging)

The peaks in transition 0 of 32 samples have been aligned.
The peaks in transition 1 of 32 samples have been aligned.
The peaks in transition 2 of 32 samples have been aligned.
The peaks in transition 3 of 32 samples have been aligned.
The peaks in transition 4 of 32 samples have been aligned.
The peaks in transition 5 of 32 samples have been aligned.
The peaks in transition 6 of 32 samples have been aligned.
The peaks in transition 7 of 32 samples have been aligned.
The peaks in transition 8 of 32 samples have been aligned.
The peaks in transition 9 of 32 samples have been aligned.
The peaks in transition 10 of 32 samples have been aligned.
The peaks in transition 11 of 32 samples have been aligned.
The peaks in transition 12 of 32 samples have been aligned.
The peaks in transition 13 of 32 samples have been aligned.
The peaks in transition 14 of 32 samples have been aligned.
The peaks in transition 15 of 32 samples have been aligned.
The peaks in transition 16 of 32 samples have been

In [ ]:
# Aligned result after preliminary correction
preliminarily_corrected_and_aligned_peaks.to_csv('preliminary_correction_and_alignment.csv')
preliminarily_corrected_and_aligned_peaks

index  1_time  ...    32_area  32_time_4markers
Transition Label                 ...                             
0          A          0   10.88  ...  6642411.0         12.628924
1          A          0   11.13  ...  2675259.0         12.810091
           B          1     NaN  ...   104233.0         14.011729
2          A          0    5.55  ...    44876.0          5.779977
3          A          0   10.63  ...   290774.0         11.905808
...                 ...     ...  ...        ...               ...
264        A          0     NaN  ...        NaN               NaN
           B          0     NaN  ...        NaN               NaN
265        A          0     NaN  ...        NaN               NaN
           B          1     NaN  ...        NaN               NaN
           C          2     NaN  ...   371400.0         26.911170

[777 rows x 97 columns]

In [ ]:
# Obtain preliminarily aligned peaks in Intermediate Stage
def select_peaks_inIntermediateStage(df,m2_transition,m3_transition):
    num=int(df.shape[1]/3)

    #To avoid modifying input parameter
    df_inIntermediateStage=df.copy()
    for i in range(1,num+1):
        df_inIntermediateStage[str(i)+'_time']=df_inIntermediateStage[str(i)+'_time_4markers']
        df_inIntermediateStage=df_inIntermediateStage.drop(str(i)+'_area',axis='columns')

    #With a duplicate of corrected time, the calculated average and std do not change
    df_inIntermediateStage['Average_MT']=df_inIntermediateStage.mean(axis=1)
    df_inIntermediateStage['std']=df_inIntermediateStage.drop('Average_MT',axis='columns').std(axis=1,ddof=0)
    df_inIntermediateStage['RSD(%)']=df_inIntermediateStage['std']/df_inIntermediateStage['Average_MT']*100
    df_inIntermediateStage=df_inIntermediateStage.drop(['std'],axis=1)

    for i in range(1,num+1):
        df_inIntermediateStage[str(i)+'_time']=df[str(i)+'_time']

    df_inIntermediateStage=df_inIntermediateStage.sort_values(by='Average_MT').reset_index().set_index(['Transition'])
    df_inIntermediateStage=df_inIntermediateStage.loc[(df_inIntermediateStage['Average_MT']>=df_inIntermediateStage['Average_MT'][m2_transition])&\
    (df_inIntermediateStage['Average_MT']<=df_inIntermediateStage['Average_MT'][m3_transition])]
    df_inIntermediateStage=df_inIntermediateStage.reset_index().set_index(['Transition','Label'])
    return df_inIntermediateStage

In [ ]:
preliminarily_aligned_inIntermediateStage = select_peaks_inIntermediateStage(preliminarily_corrected_and_aligned_peaks,m2_transition,m3_transition)
preliminarily_aligned_inIntermediateStage

,,index,1_time,1_time_4markers,2_time,2_time_4markers,3_time,3_time_4markers,4_time,4_time_4markers,5_time,5_time_4markers,6_time,6_time_4markers,7_time,7_time_4markers,8_time,8_time_4markers,9_time,9_time_4markers,10_time,10_time_4markers,11_time,11_time_4markers,12_time,12_time_4markers,13_time,13_time_4markers,14_time,14_time_4markers,15_time,15_time_4markers,16_time,16_time_4markers,17_time,17_time_4markers,18_time,18_time_4markers,19_time,19_time_4markers,20_time,20_time_4markers,21_time,21_time_4markers,22_time,22_time_4markers,23_time,23_time_4markers,24_time,24_time_4markers,25_time,25_time_4markers,26_time,26_time_4markers,27_time,27_time_4markers,28_time,28_time_4markers,29_time,29_time_4markers,30_time,30_time_4markers,31_time,31_time_4markers,32_time,32_time_4markers,Average_MT,RSD(%)
Transition,Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24,A,0,12.03,13.789062,15.80,13.789062,11.47,13.789062,12.75,13.789063,14.00,13.789062,13.84,13.789063,15.15,13.789062,12.66,13.789063,13.03,13.789062,15.59,13.789063,14.59,13.789062,13.87,13.789062,14.50,13.789062,12.81,13.789062,13.53,13.789062,11.38,13.789063,13.06,13.789062,15.68,13.789062,16.18,13.789063,14.00,13.789062,14.59,13.789062,12.75,13.789062,15.62,13.789063,11.28,13.789062,13.03,13.789062,14.12,13.789062,14.90,13.789062,12.97,13.789063,13.93,13.789062,13.93,13.789062,14.12,13.789062,14.09,13.789062,13.576923,12.500000
86,B,0,12.22,14.039986,16.15,14.052631,11.47,13.789062,12.81,13.857438,14.09,13.872977,13.81,13.757160,14.93,13.601387,12.81,13.969255,13.03,13.789062,15.77,13.929765,14.65,13.842342,13.78,13.700634,14.74,14.003808,13.00,14.016269,13.59,13.850519,11.38,13.789063,13.09,13.823616,15.84,13.914227,16.46,14.002172,14.12,13.903950,14.28,13.513363,12.87,13.931661,15.80,13.929040,11.25,13.747671,13.19,13.955854,14.28,13.941779,14.74,13.649021,13.16,14.004207,13.97,13.830845,13.93,13.789062,14.15,13.817953,14.09,13.789062,13.643226,12.533840
259,D,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.79,20.551060,13.700707,70.710678
103,C,0,NaN,NaN,16.71,14.471097,NaN,NaN,NaN,NaN,NaN,NaN,14.34,14.322142,NaN,NaN,13.22,14.464005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.09,14.362759,NaN,NaN,NaN,NaN,16.43,14.373367,17.11,14.493916,NaN,NaN,15.09,14.232256,13.22,14.349076,16.37,14.369806,11.69,14.357609,NaN,NaN,NaN,NaN,NaN,NaN,13.40,14.276547,14.21,14.081815,14.43,14.312289,14.59,14.241486,14.50,14.204220,13.865316,18.272884
160,C,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.36,18.058574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.18,16.696418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.901997,50.191645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,D,0,NaN,NaN,27.12,24.286566,NaN,NaN,NaN,NaN,24.00,23.753593,NaN,NaN,NaN,NaN,22.98,24.313166,NaN,NaN,26.47,23.971868,NaN,NaN,NaN,NaN,25.06,24.022438,22.76,23.913438,23.19,23.622907,19.83,26.434461,NaN,NaN,26.43,23.808553,27.25,24.095640,23.94,23.751719,NaN,NaN,23.13,24.219608,NaN,NaN,NaN,NaN,NaN,NaN,24.50,24.133140,NaN,NaN,22.66,23.766774,24.00,24.122417,NaN,NaN,NaN,NaN,NaN,NaN,23.368793,18.458005
219,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.56,26.293679,NaN,NaN,NaN,NaN,24.85,26.396107,NaN,NaN,22.45,26.246018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06,26.339077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.394418,35.356046
123,D,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
# Function 6
# Filter out possible markers N1 and N2 in Intermediate Stage
# based on one condition: this analyte can be found in all samples
def find_markers_inIntermediateStage(df_inIntermediateStage_input,chart_from_idx_to_transition_and_label_inIntermediateStage,m2_transition,m3_transition):

    #To avoid modifying input parameter
    df_inIntermediateStage=df_inIntermediateStage_input.copy()

    #Select m2_idx and m3_idx representing peak in m2_transition and m3_transition (its label must be 'A')
    m2_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,m2_transition,'A')
    m3_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,m3_transition,'A')
    if m2_idx==-1 or m3_idx==-1:
        return

    #Rename columns to temporarily fit in with function2
    for i in range(1,len(df_inIntermediateStage)+1):
        df_inIntermediateStage[i]=df_inIntermediateStage[i].drop(columns=['Transition','Label']).rename_axis('Transition',axis='rows').reset_index()#possible danger: label may be dropped

    #Find potential markers, replace temporary variable 'idx' and print
    potential_markers_inIntermediateStage=F2_find_markers(df_inIntermediateStage,False)
    potential_markers_inIntermediateStage=potential_markers_inIntermediateStage.reset_index(drop=True)
    potential_markers_inIntermediateStage=potential_markers_inIntermediateStage.rename(columns={'Transition':'idx'})
    potential_markers_inIntermediateStage['Transition']=\
        chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][potential_markers_inIntermediateStage['idx']].reset_index(drop=True)
    potential_markers_inIntermediateStage['Label']=\
        chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][potential_markers_inIntermediateStage['idx']].reset_index(drop=True)
    potential_markers_inIntermediateStage=\
        potential_markers_inIntermediateStage.set_index('idx')
    cols=potential_markers_inIntermediateStage.columns.tolist()
    cols=cols[-2:]+cols[:-2]
    potential_markers_inIntermediateStage=potential_markers_inIntermediateStage[cols]
    print('Migration time markers for Intermediate Stage may be selected from the following table:')
    print('')
    print(potential_markers_inIntermediateStage.set_index(['Transition','Label'],drop=True))
    return potential_markers_inIntermediateStage

#Further correct the migration time between marker 2 and marker 3
#There should be only single peak in m2_transition and m3_transiton, respectively
def correcting_by4markers_byEnumeration_inIntermediateStage(dfs,chart_from_idx_to_transition_and_label_inIntermediateStage,\
    potential_markers_inIntermediateStage,m2_transition,m3_transition):

    #Select m2_idx and m3_idx representing peak in m2_transition and m3_transition (its label must be 'A')
    m2_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,m2_transition,'A')
    m3_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,m3_transition,'A')
    if m2_idx==-1 or m3_idx==-1:
        return

    #Count the loops for enumeration to show progress later in real time
    cnt0=0
    for n1_idx in potential_markers_inIntermediateStage.index:
        if n1_idx in [m2_idx,m3_idx]:
            continue
        for n2_idx in potential_markers_inIntermediateStage.index:
            if n2_idx in [m2_idx,n1_idx,m3_idx]:
                continue
            if potential_markers_inIntermediateStage['Average_MT'][n1_idx]>=potential_markers_inIntermediateStage['Average_MT'][n2_idx]:
                continue
            cnt0+=1
    print(str(cnt0)+' loops will be run for enumeration')

    #Enumerate
    cnt=0
    min_avg_rsd=2**31
    best_corrected_dfs=None
    best_idx2=0
    best_idx3=0

    for n1_idx in potential_markers_inIntermediateStage.index:
        if n1_idx in [m2_idx,m3_idx]:
            continue
        for n2_idx in potential_markers_inIntermediateStage.index:
            if n2_idx in [m2_idx,n1_idx,m3_idx]:
                continue
            if potential_markers_inIntermediateStage['Average_MT'][n1_idx]>=potential_markers_inIntermediateStage['Average_MT'][n2_idx]:
                continue
            print()
            cnt+=1
            print('Progress: '+str(cnt)+'/'+str(cnt0))
            print('Select these peaks as markers: '+\
                str(m2_transition)+'A, '+\
                str(chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][n1_idx])+\
                chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][n1_idx]+', '+\
                str(chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][n2_idx])+\
                chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][n2_idx]+', '+\
                str(m3_transition)+'A')
            
            going_to_continue=False
            for i in range(1,len(dfs)+1):
                if potential_markers_inIntermediateStage[str(i)+'_time'][m2_idx]>potential_markers_inIntermediateStage[str(i)+'_time'][n1_idx]:
                    print('In sample '+str(i)+', the MT of M2 is greater than N1, migration time cannot be corrected by these markers')
                    going_to_continue=True
                if potential_markers_inIntermediateStage[str(i)+'_time'][n1_idx]>potential_markers_inIntermediateStage[str(i)+'_time'][n2_idx]:
                    print('In sample '+str(i)+', the MT of N1 is greater than N2, migration time cannot be corrected by these markers')
                    going_to_continue=True
                if potential_markers_inIntermediateStage[str(i)+'_time'][n2_idx]>potential_markers_inIntermediateStage[str(i)+'_time'][m3_idx]:
                    print('In sample '+str(i)+', the MT of N2 is greater than M3, migration time cannot be corrected by these markers')
                    going_to_continue=True
            if going_to_continue:
                continue
            
            #Rename columns to temporarily fit in with function2
            potential_markers_inIntermediateStage_for_fun2=\
            potential_markers_inIntermediateStage.drop(['Transition','Label'],axis='columns')\
                .reset_index().rename(columns={'idx':'Transition'})
            dfs_for_fun2={}
            for i in range(1,len(dfs)+1):
                dfs_for_fun2[i]=dfs[i].reset_index().drop(['Transition','Label'],axis='columns').\
                rename(columns={'idx':'Transition'})
            
            dfs_temp=correcting_by4markers(dfs_for_fun2,potential_markers_inIntermediateStage_for_fun2,m2_idx,n1_idx,n2_idx,m3_idx,False)
            if dfs_temp==-1:
                print('Migration time cannot be corrected by these markers')
                continue
            
            for i in range(1,len(dfs)+1):
                dfs_temp[i]=dfs_temp[i].set_index('Transition',drop=True).rename_axis('idx',axis='rows')

            corrected_dfs=dfs_temp[1]
            for i in range(2,len(dfs)+1):
                corrected_dfs=corrected_dfs.join(dfs_temp[i],how='outer')

            corrected_dfs_temp=corrected_dfs.copy()
            for i in range(1,len(dfs)+1):
                corrected_dfs_temp=corrected_dfs_temp.drop([str(i)+'_time',str(i)+'_area'],axis='columns')
            corrected_dfs['Average_MT']=corrected_dfs_temp.mean(axis=1)
            corrected_dfs['std']=corrected_dfs_temp.std(axis=1,ddof=0)
            corrected_dfs['RSD(%)']=corrected_dfs['std']/corrected_dfs['Average_MT']*100
            corrected_dfs=corrected_dfs.drop(['std'], axis=1)
            corrected_dfs=corrected_dfs.reset_index(drop=False)
            corrected_dfs['Transition']=\
            chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][corrected_dfs['idx']].reset_index(drop=True)
            corrected_dfs['Label']=\
                chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][corrected_dfs['idx']].reset_index(drop=True)
            corrected_dfs=corrected_dfs.set_index('idx')
            cols=corrected_dfs.columns.tolist()
            cols=cols[-2:]+cols[:-2]
            corrected_dfs=corrected_dfs[cols]
            avg_rsd=corrected_dfs['RSD(%)'].mean()
            print('The result of migration time correction:')
            print(corrected_dfs.set_index(['Transition','Label'],drop=True))
            print('The average value of RSD for each peak is: '+str(avg_rsd)+'%')
            if avg_rsd<min_avg_rsd:
                min_avg_rsd=avg_rsd
                best_corrected_dfs=corrected_dfs
                best_n1_idx=n1_idx
                best_n2_idx=n2_idx

    best_corrected_dfs=best_corrected_dfs.set_index(['Transition','Label'],drop=True)
    n1_transition=chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][best_n1_idx]
    n1_label=chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][best_n1_idx]
    n2_transition=chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][best_n2_idx]
    n2_label=chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][best_n2_idx]
    print('')
    print('The best markers for correction are '+\
        str(m2_transition)+'A, '+str(n1_transition)+n1_label+', '+str(n2_transition)+n2_label+', '+str(m3_transition)+'A')
    print('The best result of migration time correction:')
    print(best_corrected_dfs)
    print('The average value of RSD for each peak is: '+str(min_avg_rsd)+'%')
    return best_corrected_dfs,n1_transition,n1_label,n2_transition,n2_label

In [ ]:
# Function 6 is in this code cell
chart_from_idx_to_transition_and_label_inIntermediateStage = preliminarily_aligned_inIntermediateStage.reset_index().loc[:,['Transition','Label']]\
    .sort_values(by=['Transition','Label']).reset_index(drop=True).rename_axis('idx', axis='rows')

# Replace columns 'Transition' and 'Label' by 'idx' in 'preliminarily_corrected_and_aligned_peaks' in to make Function 5 applicable for the result
dfs_inIntermediateStage = {}
for i in range(1,len(raw_data)+1):
    dfs_inIntermediateStage[i] = pd.DataFrame({'Transition':[],'Label':[],str(i)+'_time':[], str(i)+'_area':[]})
    for j in range(0,chart_from_idx_to_transition_and_label_inIntermediateStage.shape[0]):
        new_transition=int(chart_from_idx_to_transition_and_label_inIntermediateStage['Transition'][j])
        new_label=chart_from_idx_to_transition_and_label_inIntermediateStage['Label'][j]
        new_peak=pd.DataFrame({'Transition':[new_transition],'Label':[new_label],\
            str(i)+'_time':[preliminarily_corrected_and_aligned_peaks[str(i)+'_time'][(new_transition,new_label)]],\
            str(i)+'_area':[preliminarily_corrected_and_aligned_peaks[str(i)+'_area'][(new_transition,new_label)]]},\
            index=[j])
        dfs_inIntermediateStage[i]=dfs_inIntermediateStage[i].append(new_peak)
    dfs_inIntermediateStage[i]=dfs_inIntermediateStage[i].rename_axis('idx',axis='rows').dropna(axis='index',how='any').astype({'Transition': 'int'})

potential_markers_inIntermediateStage=find_markers_inIntermediateStage(dfs_inIntermediateStage,chart_from_idx_to_transition_and_label_inIntermediateStage,m2_transition,m3_transition)

# Function 6
final_dfs_inIntermediateStage,n1_transition,n1_label,n2_transition,n2_label=correcting_by4markers_byEnumeration_inIntermediateStage(dfs_inIntermediateStage,\
    chart_from_idx_to_transition_and_label_inIntermediateStage,potential_markers_inIntermediateStage,m2_transition,m3_transition)

All 14 one-peak transitions were selected.

No peaks are checked and removed, for variable "check_and_correct" is set as False.

Function 2 is completed.

Migration time markers for Intermediate Stage may be selected from the following table:

                  1_time  2_time  3_time  ...  32_time  Average_MT     RSD(%)
Transition Label                          ...                                
24         A       12.03   15.80   11.47  ...    14.09   13.789062   9.301386
86         B       12.22   16.15   11.47  ...    14.09   13.857812   9.449480
147        A       14.50   19.95   13.28  ...    16.24   16.553750  11.382868
206        B       18.52   23.29   17.21  ...    20.67   20.750312   7.948570
230        A       18.11   23.07   17.43  ...    21.14   20.829375   7.860998
262        A       18.55   23.38   17.96  ...    21.39   21.122188   7.653165
260        A       19.05   23.66   18.45  ...    21.76   21.452188   7.490496
258        A       18.92   23.82   18.52  ...    21.79

In [ ]:
final_dfs_inIntermediateStage

,,1_time,1_area,1_time_4markers,2_time,2_area,2_time_4markers,3_time,3_area,3_time_4markers,4_time,4_area,4_time_4markers,5_time,5_area,5_time_4markers,6_time,6_area,6_time_4markers,7_time,7_area,7_time_4markers,8_time,8_area,8_time_4markers,9_time,9_area,9_time_4markers,10_time,10_area,10_time_4markers,11_time,11_area,11_time_4markers,12_time,12_area,12_time_4markers,13_time,13_area,13_time_4markers,14_time,...,20_area,20_time_4markers,21_time,21_area,21_time_4markers,22_time,22_area,22_time_4markers,23_time,23_area,23_time_4markers,24_time,24_area,24_time_4markers,25_time,25_area,25_time_4markers,26_time,26_area,26_time_4markers,27_time,27_area,27_time_4markers,28_time,28_area,28_time_4markers,29_time,29_area,29_time_4markers,30_time,30_area,30_time_4markers,31_time,31_area,31_time_4markers,32_time,32_area,32_time_4markers,Average_MT,RSD(%)
Transition,Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,B,NaN,NaN,NaN,31.83,934.0,28.3092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.74,933.0,28.712,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.510586,0.706438
5,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.39,7485.0,29.3669,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.366879,0.000000
8,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.58,8415.0,33.9463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.946339,0.000000
13,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.23,...,NaN,NaN,NaN,NaN,NaN,20.04,26180.0,20.8294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.26,31735.0,21.032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.92,42080.0,20.6435,20.876216,0.741761
23,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.71,21506.0,29.6897,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.689733,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.79,17765.0,20.5335,20.533459,0.000000
260,A,19.05,104005.0,21.7411,23.66,31756.0,21.3403,18.45,28019.0,21.7664,20.42,24323.0,21.3518,21.67,31822.0,21.3425,21.48,69610.0,21.5146,23.29,17746.0,21.3923,20.29,44832.0,21.2165,20.42,69390.0,21.3463,23.63,20548.0,21.4675,22.38,38337.0,21.3762,21.54,51371.0,21.4379,22.51,92930.0,21.5091,20.67,...,13075.0,21.4897,22.69,83259.0,21.4832,20.70,127025.0,21.55,23.75,18680.0,21.4293,17.89,36426.0,21.6149,20.67,50660.0,21.4151,21.85,27230.0,21.4039,22.94,84995.0,21.4255,20.57,32690.0,21.3818,21.60,154400.0,21.5548,21.54,159400.0,21.4526,21.79,156391.0,21.4968,21.76,279500.0,21.4718,21.4499

In [ ]:
# Function 7
# Correct the migration times by using 6 markers
# If migration times cannot be corrected by these markers due to gamma equals 0, return -1
def correcting_by6markers(dfs_input,dfs_inIntermediateStage_input,df_std_input,df_std_inIntermediateStage_input,chart_from_idx_to_transition_and_label_inIntermediateStage,\
    m1_transition,m2_transition,n1_transition,n1_label,n2_transition,n2_label,m3_transition,m4_transition):

    #Select m2_idx and m3_idx representing peak in m2_transition and m3_transition (its label must be 'A')
    m2_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,m2_transition,'A')
    n1_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,n1_transition,n1_label)
    n2_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,n2_transition,n2_label)
    m3_idx=transition_and_label_to_idx(chart_from_idx_to_transition_and_label_inIntermediateStage,m3_transition,'A')

    dfs=dfs_input.copy()
    df_std=df_std_input.copy()
    df_std_temp = df_std.set_index('Transition')

    #Temporarily fit in with function get_gamma
    dfs_inIntermediateStage=dfs_inIntermediateStage_input.copy()
    df_std_inIntermediateStage=df_std_inIntermediateStage_input.reset_index().drop(['Transition','Label'],axis='columns').\
        rename(columns={'idx':'Transition'})
    df_std_inIntermediateStage.index+=1
    df_std_inIntermediateStage_temp = df_std_inIntermediateStage.set_index('Transition')

    tm1_std = df_std_temp.loc[m1_transition,'Average_MT']
    tm2_std = df_std_temp.loc[m2_transition,'Average_MT']
    tm3_std = df_std_temp.loc[m3_transition,'Average_MT']
    tn1_std = df_std_inIntermediateStage_temp.loc[n1_idx,'Average_MT']
    tn2_std = df_std_inIntermediateStage_temp.loc[n2_idx,'Average_MT']
    for i in range(1,len(dfs)+1):

        #Temporarily fit in with function get_gamma
        dfs_inIntermediateStage[i]=dfs_inIntermediateStage[i].reset_index().drop(['Transition','Label'],axis='columns').\
            rename(columns={'idx':'Transition'})

        gamma1 = get_gamma(dfs, i, df_std, m1_transition, m2_transition)
        gamma2 = get_gamma(dfs, i, df_std, m3_transition, m4_transition)
        gamma_n1 = get_gamma(dfs_inIntermediateStage, i, df_std_inIntermediateStage, m2_idx, n1_idx)
        gamma_n2 = get_gamma(dfs_inIntermediateStage, i, df_std_inIntermediateStage, n1_idx, n2_idx)
        gamma_n3 = get_gamma(dfs_inIntermediateStage, i, df_std_inIntermediateStage, n2_idx, m3_idx)

        if gamma1==0:
            print('Migration times cannot be corrected by m1_transition='+str(m1_transition)+\
                ' and m2_transition='+str(m2_transition)+', because their migration '+\
                'times in sample '+str(i)+' are identical.')
            return -1
        if gamma2==0:
            print('Migration times cannot be corrected by m3_transition='+str(m3_transition)+\
                  ' and m4_transition='+str(m4_transition)+', because their migration '+\
                  'times in sample '+str(i)+' are identical.')
            return -1
        if gamma_n1==0:
            print('Migration times cannot be corrected by m2_transition='+str(m2_transition)+\
                ' and n1='+str(n1_transition)+n1_label+', because their migration '+\
                'times in sample '+str(i)+' are identical.')
            return -1
        if gamma_n2==0:
            print('Migration times cannot be corrected by n1='+str(n1_transition)+n1_label+\
                ' and n2='+str(n2_transition)+n2_label+', because their migration '+\
                'times in sample '+str(i)+' are identical.')
            return -1
        if gamma_n3==0:
            print('Migration times cannot be corrected by n2='+str(n2_transition)+n2_label+\
                ' and m3_transition='+str(m3_transition)+', because their migration '+\
                'times in sample '+str(i)+' are identical.')
            return -1

        df_temp = dfs[i].set_index('Transition')
        #Replace temporary axis name by real name 'idx'
        dfs_inIntermediateStage_temp=dfs_inIntermediateStage[i].set_index('Transition').rename_axis('idx',axis='rows')

        tm1 = df_temp.loc[m1_transition,str(i)+'_time']
        tm2 = df_temp.loc[m2_transition,str(i)+'_time']
        tm3 = df_temp.loc[m3_transition,str(i)+'_time']
        tn1 = dfs_inIntermediateStage_temp.loc[n1_idx,str(i)+'_time']
        tn2 = dfs_inIntermediateStage_temp.loc[n2_idx,str(i)+'_time']
        dfs[i][str(i)+'_time_6markers']= None
        analyte_num = dfs[i].shape[0]
        for j in range(0,analyte_num):
            t_origin = dfs[i].loc[j,str(i)+'_time']
            if t_origin <= tm2:
                t_6marker = 1/((1/tm1_std)-(1/gamma1)*((1/tm1)-(1/t_origin)))
            elif t_origin <= tn1:
                t_6marker = 1/((1/tm2_std)-(1/gamma_n1)*((1/tm2)-(1/t_origin)))
            elif t_origin <= tn2:
                t_6marker = 1/((1/tn1_std)-(1/gamma_n2)*((1/tn1)-(1/t_origin)))
            elif t_origin <= tm3:
                t_6marker = 1/((1/tn2_std)-(1/gamma_n3)*((1/tn2)-(1/t_origin)))
            else:
                t_6marker = 1/((1/tm3_std)-(1/gamma2)*((1/tm3)-(1/t_origin)))
            dfs[i].loc[j,str(i)+'_time_6markers'] = t_6marker
    return dfs

In [ ]:
# Using Function 7 for the final correction
# Apply Function 4 for preliminary correction
finally_corrected_peaks=correcting_by6markers(raw_data,dfs_inIntermediateStage,potential_markers,potential_markers_inIntermediateStage,chart_from_idx_to_transition_and_label_inIntermediateStage,\
    m1_transition,m2_transition,n1_transition,n1_label,n2_transition,n2_label,m3_transition,m4_transition)

# Using Function 5 for the alignment after preliminary correction
finally_aligned_peaks_before_reexamination = align(finally_corrected_peaks, max_rsd_of_finally_aligned_peaks, 0)

The peaks in transition 0 of 32 samples have been aligned.
The peaks in transition 1 of 32 samples have been aligned.
The peaks in transition 2 of 32 samples have been aligned.
The peaks in transition 3 of 32 samples have been aligned.
The peaks in transition 4 of 32 samples have been aligned.
The peaks in transition 5 of 32 samples have been aligned.
The peaks in transition 6 of 32 samples have been aligned.
The peaks in transition 7 of 32 samples have been aligned.
The peaks in transition 8 of 32 samples have been aligned.
The peaks in transition 9 of 32 samples have been aligned.
The peaks in transition 10 of 32 samples have been aligned.
The peaks in transition 11 of 32 samples have been aligned.
The peaks in transition 12 of 32 samples have been aligned.
The peaks in transition 13 of 32 samples have been aligned.
The peaks in transition 14 of 32 samples have been aligned.
The peaks in transition 15 of 32 samples have been aligned.
The peaks in transition 16 of 32 samples have been

In [ ]:
finally_aligned_peaks_before_reexamination

index  1_time  ...    32_area  32_time_6markers
Transition Label                 ...                             
0          A          0   10.88  ...  6642411.0         12.628924
1          A          0   11.13  ...  2675259.0         12.810091
           B          0     NaN  ...        NaN               NaN
           C          0     NaN  ...   104233.0         14.061785
           D          0     NaN  ...        NaN               NaN
...                 ...     ...  ...        ...               ...
264        B          0     NaN  ...        NaN               NaN
265        A          0     NaN  ...        NaN               NaN
           B          0     NaN  ...        NaN               NaN
           C          0     NaN  ...        NaN               NaN
           D          0     NaN  ...   371400.0         26.911170

[870 rows x 97 columns]

In [ ]:
# Re-examination of the alignment results
finally_aligned_peaks_before_reexamination = with_avgMT_and_RSD(finally_aligned_peaks_before_reexamination)
finally_aligned_peaks = check_and_correct(finally_aligned_peaks_before_reexamination,allowed_EORMT,'step_last')
finally_aligned_peaks = with_avgMT_and_RSD(finally_aligned_peaks)

Peak 26A in sample 1 is abnormal, EORMT is 7.15%>2.5% to the left peak (211A) and 9.62%>2.5% to the right peak (204A).
Peak 174B in sample 1 is abnormal, EORMT is 6.39%>2.5% to the left peak (203A) and 6.50%>2.5% to the right peak (60A).
Peak 145A in sample 1 is abnormal, EORMT is 3.28%>2.5% to the left peak (36A) and 5.03%>2.5% to the right peak (39A).
Peak 166A in sample 1 is abnormal, EORMT is 2.72%>2.5% to the left peak (36A) and 4.46%>2.5% to the right peak (39A).
Peak 74A in sample 1 is abnormal, EORMT is 2.67%>2.5% to the left peak (36A) and 4.41%>2.5% to the right peak (39A).
Peak 109A in sample 1 is abnormal, EORMT is 5.37%>2.5% to the left peak (62A) and 4.30%>2.5% to the right peak (46F).
Peak 256A in sample 1 is abnormal, EORMT is 3.53%>2.5% to the left peak (147A) and 5.49%>2.5% to the right peak (110A).
Peak 73B in sample 1 is abnormal, EORMT is 4.25%>2.5% to the left peak (262A) and 3.53%>2.5% to the right peak (260A).
Peak 90B in sample 1 is abnormal, EORMT is 3.18%>2.5

In [ ]:
# Aligned result after final correction and re-examination
# A triplet of columns represents the raw migration time, peak area and corrected migration time of the peaks in a sample
finally_aligned_peaks.to_csv('final_correction_and_alignment.csv')
finally_aligned_peaks

1_time      1_area  ...  Average_MT    RSD(%)
Transition Label                      ...                      
0          A       10.88  11070312.0  ...   12.489825  0.835344
1          A       11.13   1100433.0  ...   12.736012  0.827347
           B         NaN         NaN  ...   13.612400  2.424721
           C         NaN         NaN  ...   15.996290  0.000000
2          A        5.55     17784.0  ...    5.833223  1.038241
...                  ...         ...  ...         ...       ...
264        B         NaN         NaN  ...   27.011740  0.000000
265        A         NaN         NaN  ...   10.789401  1.757105
           B         NaN         NaN  ...   12.689129  0.000000
           C         NaN         NaN  ...   16.875601  0.000000
           D         NaN         NaN  ...   26.962306  0.438601

[874 rows x 98 columns]